# Kurs 8

Nachdem wir uns in den vergangenen Kursen mit vielen Programmierkonzepten näher beschäftigt haben (allen voran Typen und Makros), wollen wir nun ein praxisnahes Beispiel betrachten, welches ein bisschen die verschiedenen Fäden zusammenführt. Das Setting ist: Definiere Arithmetik für verschiedene Temparatureinheiten (von [hier](https://discourse.julialang.org/t/promotion-multiple-dispatch-in-example-using-temperature/50843) geklaut).

## Ein fancy Beispiel

In [ ]:
# Wir wollen zu diesen Funktionen (die bereits ab Werk in Julia sind, deshalb im Base module) neue Funktionalität hinzufügen.
import Base: +, -, *, promote, promote_rule, convert, show

# definiere neuen Typ
abstract type Temperature end
types = [:Celsius, :Kelvin, :Fahrenheit]

# lege für jede Temperatur interne Daten an und definiere Addition/Subtraktion gleicher Temperaturen
for T in types
    # @eval begin ... end analog zu eval(...)
    @eval begin
        # $T kopiert die entsprechende Temperatur sozusagen als Teil des Codes an die Stelle nach struct,
        # damit wir diesen Block nicht mehrfach schreiben müssen
        # Das ist metaprogramming!
        struct $T <: Temperature
            value::Float64
        end

        +(x::$T, y::$T) = $T(x.value + y.value)
        -(x::$T, y::$T) = $T(x.value - y.value)
    end
end

In [ ]:
Celsius(1.0)

In [ ]:
+(Celsius(1.0), Celsius(2.0))

In [ ]:
Celsius(1.0) + Celsius(2.0)

In [ ]:
# Lege fest, wie t in erstes Argument umgwandelt wird.
convert(::Type{Kelvin}, t::Celsius) = Kelvin(t.value + 273.15)
convert(::Type{Kelvin}, t::Fahrenheit) = Kelvin(Celsius(t))
convert(::Type{Celsius}, t::Kelvin) = Celsius(t.value - 273.15)
convert(::Type{Celsius}, t::Fahrenheit) = Celsius((t.value - 32) * 5 / 9)

# Wenn wir wollen, können wir auch conversion in Fahrenheit definieren
# convert(::Type{Fahrenheit}, t::Celsius) = Fahrenheit(t.value*9/5 + 32)
# convert(::Type{Fahrenheit}, t::Kelvin) = Fahrenheit(Ceslius(t))

In [ ]:
convert(Kelvin, Celsius(1.0))

In [ ]:
# Baue die obigen convert-Funktionen in die Konstruktoren ein.
for T in types, S in types
    if S != T
        @eval $T(temp::$S) = convert($T, temp)
    end
end

In [ ]:
# Wir bekommen dieses nette Verhalten (Initialisierung mit anderen Temperaturen ist möglich) 
Kelvin(Celsius(1.0))

Mit *promotion* bezeichnet man die *conversion* von gemischten Typen zu einem gemeinsamen Typ:

In [ ]:
# Gegeben zwei Einheiten: Welche hätten wir lieber?
promote_rule(::Type{Kelvin}, ::Type{Celsius}) = Kelvin
promote_rule(::Type{Fahrenheit}, ::Type{Kelvin}) = Kelvin
promote_rule(::Type{Fahrenheit}, ::Type{Celsius}) = Celsius

In [ ]:
promote_type(Kelvin, Celsius)

In [ ]:
promote(Kelvin(1.0), Celsius(1.0))

In [ ]:
# definiere Arithmetik für unterschiedliche Temperatur-structs
+(x::Temperature, y::Temperature) = +(promote(x, y)...);
-(x::Temperature, y::Temperature) = -(promote(x, y)...);

In [ ]:
Fahrenheit(4) + Celsius(5)

Was passiert hier? Zunächst wird durch promotion gefragt, in welchem Typ wir gerne rechnen würden. Entsprechend werden beide Inputs dorthin konvertiert. Für jeden einzelnen Typ ist Arithmetik aber bereits wohldefiniert, also bekommen wir etwas Sinnvolles raus.

In [ ]:
# Wir wollen am Ende schreiben können: 2°K anstelle Kelvin(2)
abstract type TemperatureSymbol end
# Lege für jede Temperatur einen zusätzlichen struct an, damit dieser als repräsentatives Symbol benutzt werden kann
# ist nicht SI-konform, aber nvm
symbols = Symbol.(["°C", "°K", "°F"])
for i in 1:length(symbols)
    @eval begin
        struct $(symbols[i]) <: TemperatureSymbol
        end
        *(x::Real, y::Type{$(symbols[i])}) = $(types[i])(x)
    end
end

In [ ]:
Kelvin(2)

Folgendes soll fehlschlagen; würde man sich den struct ```TemperatureSymbol``` sparen und direkt die Multiplikation auf Temperaturen definieren, dann hätte man ```const °C = Celsius(1)``` und man hätte diesen Sicherheitsmechanismus nicht (Operation würde ungewünscht funktionieren).

In [ ]:
2°C + °C

In [ ]:
0°C + 0°K

Nun implementieren wir noch eine hübschere Ausgabe:

In [ ]:
# custom pretty-printing (nutze Base.show)
for T in types
    # Die Benennung der Ausgabe ist hier auch ein wieder ein bisschen arbiträr, nicht SI-konform
    @eval show(io::IO, x::$T) = println("$(x.value) °" * $(String(T)))
end
Kelvin(1)